In [3]:
from pyspark.sql import SparkSession

spark = SparkSession\
            .builder.master('yarn').appName('prueba').enableHiveSupport().getOrCreate()

In [36]:
usuario = spark.read.json('s3://bigdata-desafio/yelp-data/user.json')

In [37]:
usuario.take(1)

[Row(average_stars=4.03, compliment_cool=1, compliment_cute=0, compliment_funny=1, compliment_hot=2, compliment_list=0, compliment_more=0, compliment_note=1, compliment_photos=0, compliment_plain=1, compliment_profile=0, compliment_writer=2, cool=25, elite='2015,2016,2017', fans=5, friends='c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g, ajcnq75Z5xxkvUSmmJ1bCg, BSMAmp2-wMzCkhTfq9ToNg, jka10dk9ygX76hJG0gfPZQ, dut0e4xvme7QSlesOycHQA, l4l5lBnK356zBua7B-UJ6Q, 0HicMOOs-M_gl2eO-zES4Q, _uI57wL2fLyftrcSFpfSGQ, T4_Qd0YWbC3co6WSMw4vxg, iBRoLWPtWmsI1kdbE9ORSA, xjrUcid6Ymq0DoTJELkYyw, GqadWVzJ6At-vgLzK_SKgA, DvB13VJBmSnbFXBVBsKmDA, vRP9nQkYTeNioDjtxZlVhg, gT0A1iN3eeQ8EMAjJhwQtw, 6yCWjFPtp_AD4x93WAwmnw, 1dKzpNnib-JlViKv8_Gt5g, 3Bv4_JxHXq-gVLOxYMQX0Q, ikQyfu1iViYh8T0us7wiFQ, f1GGltNaB7K5DR1jf3dOmg, tgeFUChlh7v8bZFVl2-hjQ, -9-9oyXlqsMG2he5xIWdLQ, Adj9fBPVJad8vSs-mIP7gw, Ce49RY8CKXVsTifxRYFTsw, M1_7TLi8CbdA89nFLlH4iw, wFsNv-hqbW_F5-IRqfBN6g, 0Q1L7zXHocaUZ2gsG2XJeg, cBFgmOCBdhYa0xoFEAzp_g, VrD_AgiFvzqt

In [17]:
usuario.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)

In [164]:
from pyspark.sql.functions import when,length,split

In [134]:
from pyspark.sql.types import StructType,StructField,IntegerType,FloatType,StringType,LongType,DoubleType,ArrayType

In [198]:
schema = StructType([
    StructField('average_stars', DoubleType(), True),
    StructField('compliment_cool', LongType(), True),
    StructField('compliment_cute', LongType(), nullable= True),
    StructField('compliment_funny', LongType(), nullable= True),
    StructField('compliment_hot', LongType(), nullable= True),
    StructField('compliment_list', LongType(), nullable= True),
    StructField('compliment_more', LongType(), nullable= True),
    StructField('compliment_note', LongType(), nullable= True),
    StructField('compliment_photos', LongType(), nullable= True),
    StructField('compliment_plain', LongType(), nullable= True),
    StructField('compliment_profile', LongType(), nullable= True),
    StructField('compliment_writer', LongType(), nullable= True),
    StructField('cool', LongType(), nullable= True),
    StructField('elite', StringType(), nullable= True),
    StructField('fans', LongType(), nullable= True),
    StructField('friends', StringType(), nullable= True),
    StructField('funny',LongType(), nullable= True),
    StructField('name', StringType(), nullable= True),
    StructField('review_count', LongType(), nullable= True),
    StructField('useful', LongType(), nullable= True),
    StructField('user_id', StringType(), nullable= True),
    StructField('yelping_since', StringType(), nullable= True),
    StructField('molesto', IntegerType(), nullable= True)
])

In [199]:
usuario = spark.read.json('s3://bigdata-desafio/yelp-data/user.json',schema = schema)

In [200]:
usuario.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)
 |-- molesto: integer (nullable = true)

In [201]:
usuario = usuario\
            .withColumn('molesto',when( (usuario['average_stars']<=2) |
                                        (usuario['review_count']<=100) |
                                        (usuario['fans']==0) ,1).otherwise(0))

In [202]:
usuario.select('friends').take(1)

[Row(friends='c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g, ajcnq75Z5xxkvUSmmJ1bCg, BSMAmp2-wMzCkhTfq9ToNg, jka10dk9ygX76hJG0gfPZQ, dut0e4xvme7QSlesOycHQA, l4l5lBnK356zBua7B-UJ6Q, 0HicMOOs-M_gl2eO-zES4Q, _uI57wL2fLyftrcSFpfSGQ, T4_Qd0YWbC3co6WSMw4vxg, iBRoLWPtWmsI1kdbE9ORSA, xjrUcid6Ymq0DoTJELkYyw, GqadWVzJ6At-vgLzK_SKgA, DvB13VJBmSnbFXBVBsKmDA, vRP9nQkYTeNioDjtxZlVhg, gT0A1iN3eeQ8EMAjJhwQtw, 6yCWjFPtp_AD4x93WAwmnw, 1dKzpNnib-JlViKv8_Gt5g, 3Bv4_JxHXq-gVLOxYMQX0Q, ikQyfu1iViYh8T0us7wiFQ, f1GGltNaB7K5DR1jf3dOmg, tgeFUChlh7v8bZFVl2-hjQ, -9-9oyXlqsMG2he5xIWdLQ, Adj9fBPVJad8vSs-mIP7gw, Ce49RY8CKXVsTifxRYFTsw, M1_7TLi8CbdA89nFLlH4iw, wFsNv-hqbW_F5-IRqfBN6g, 0Q1L7zXHocaUZ2gsG2XJeg, cBFgmOCBdhYa0xoFEAzp_g, VrD_AgiFvzqtlR15vir3SQ, cpE-7HK514Sr5vpSen9CEQ, F1UYelhPFB-zIKlt0ygIZg, CQAL1hvsLMCzuJf9AglsXw, 1KnY1wr15WfEWIRLB9IS6g, QWFQ-kXBiLbid-lm5Jr3dQ, nymT8liFugCrM16lTy0ZfQ, qj69bdd885heDvUPCyHd2Q, DySCZZcgbdrlHgEovk5y9w, lZMJIDuvhT9Dy4KyquLXyA, b_9Gn7wS93AoPZPR0dIJqQ, N07g1IaLh0_6sUjtiSRe4w, Yd

In [203]:
import pyspark.sql.functions as f
usuario = usuario.withColumn('friends_count', f.size(f.split(f.col('friends'), ',')))

In [204]:
usuario.select('friends_count').show(2)

+-------------+
|friends_count|
+-------------+
|           99|
|         1152|
+-------------+
only showing top 2 rows

In [205]:
usuario = usuario.withColumn('elite_count', f.size(f.split(f.col('elite'), ',')))

In [206]:
usuario.select('elite_count').show(2)

+-----------+
|elite_count|
+-----------+
|          3|
|          1|
+-----------+
only showing top 2 rows

In [207]:
df = usuario.select(['molesto','elite_count','friends_count'])

In [211]:
names = usuario.schema.names

In [227]:
names = list(filter(lambda x: x not in ['friends','yelping_since','name','elite','user_id'],names))

In [221]:
df = usuario.select(names)

In [230]:
from pyspark.ml.feature import VectorAssembler

In [231]:
feats = df.columns

In [232]:
feats.remove('molesto')

In [235]:
assemble_feats = VectorAssembler(inputCols= feats,outputCol='assemble_feats')

In [236]:
assemble_feats = assemble_feats.transform(df)

In [239]:
assemble_feats = assemble_feats.select(['molesto','assemble_feats'])

In [240]:
train,test = assemble_feats.randomSplit([0.7,0.3])

In [242]:
from pyspark.ml.classification import LogisticRegression

In [243]:
logistic = LogisticRegression(featuresCol='assemble_feats',labelCol='molesto',predictionCol='molesto_pred')

In [247]:
logistic=logistic.fit(train)

In [249]:
from pyspark.mllib.evaluation import RegressionMetrics
def report_reg_metrics(metrics):
    """Report metrics from a RegressionMetrics object

    :metrics: a RegressionMetrics object
    :returns: a printed report.

    """

    if isinstance(metrics, RegressionMetrics) is True:
        print(f"Varianza Explicada: {round(metrics.explainedVariance, 3)}")
        print(f"Error cuadrático promedio: {round(metrics.meanSquaredError, 3)}")
        print(f"Error absoluto promedio: {round(metrics.meanAbsoluteError, 3)}")
        print(f"Raíz del error cuadrático promedio: {round(metrics.rootMeanSquaredError, 3)}")
    else:
        raise TypeError("metrics argument must be a pyspark.mllib.evaluation.RegressionMetrics object.")

In [254]:
logistic.summary.accuracy

0.9966395435220095

In [255]:
model_test= logistic.evaluate(test)

In [257]:
model_test.accuracy

0.9968180798335611

In [258]:
logistic.transform(test).show(10)

+-------+--------------------+--------------------+--------------------+------------+
|molesto|      assemble_feats|       rawPrediction|         probability|molesto_pred|
+-------+--------------------+--------------------+--------------------+------------+
|      0|(19,[0,1,3,7,12,1...|[2.76339728301244...|[0.94066553370465...|         0.0|
|      0|(19,[0,1,3,9,12,1...|[3.07991753092824...|[0.95605672002141...|         0.0|
|      0|(19,[0,1,3,12,13,...|[1.89942788959479...|[0.86982676032922...|         0.0|
|      0|(19,[0,4,7,12,13,...|[19.7136074758498...|[0.99999999725533...|         0.0|
|      0|(19,[0,4,9,11,12,...|[0.35124571694995...|[0.58691962940202...|         0.0|
|      0|(19,[0,4,9,12,13,...|[-0.5618283824888...|[0.36312451292825...|         1.0|
|      0|(19,[0,6,7,11,12,...|[0.11920464658306...|[0.52976592280258...|         0.0|
|      0|(19,[0,7,8,9,12,1...|[3.84629373335133...|[0.97908790508122...|         0.0|
|      0|(19,[0,7,8,9,12,1...|[2.93605329867057...|[0.